In [9]:
from flask import Flask, jsonify, request
from dotenv import load_dotenv
import os
import json
import requests
import pinecone

app = Flask(__name__)

# Load environment variables from .env file
load_dotenv()

# Llama API key and URL
LLAMA_API_KEY = os.getenv("OPENROUTER_API_KEY")
LLAMA_API_URL = "https://openrouter.ai/api/v1/chat/completions"  # Adjust if needed

# Pinecone API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY)

# Create or get Pinecone index
def create_or_get_index(index_name, dimension):
    existing_indexes = pinecone.list_indexes()
    if index_name not in existing_indexes:
        pinecone.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine"
        )
    return pinecone.Index(index_name)

@app.route('/update_index', methods=['POST'])
def update_index():
    # Load the review data
    with open("reviews.json") as file:
        data = json.load(file)

    # Define the index and dimension (adjust dimension if needed)
    index_name = "rag"
    dimension = 1536  # Ensure this matches the output dimension of your Llama model

    # Create or get the Pinecone index
    index = create_or_get_index(index_name, dimension)

    processed_data = []

    # Create embeddings for each review
    for review in data.get("reviews", []):
        response = requests.post(
            url=LLAMA_API_URL,
            headers={
                "Authorization": f"Bearer {LLAMA_API_KEY}",
                "Content-Type": "application/json",
            },
            data=json.dumps({
                "model": "meta-llama/llama-3.1-8b-instruct:free",  # Adjust if needed
                "messages": [
                    {"role": "user", "content": review['review']}
                ]
            })
        )
        response_data = response.json()
        embedding = response_data.get('embedding', [])  # Adjust based on actual API response

        processed_data.append(
            {
                "values": embedding,
                "id": review["professor"],
                "metadata": {
                    "review": review["review"],
                    "subject": review["subject"],
                    "stars": review["stars"],
                }
            }
        )

    # Insert the embeddings into the Pinecone index
    upsert_response = index.upsert(
        vectors=processed_data,
        namespace="ns1",
    )

    # Return a response
    return jsonify({
        "upserted_count": upsert_response['upserted_count'],
        "index_stats": index.describe_index_stats()
    })

if __name__ == '__main__':
    app.run(debug=True)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

